In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.decomposition import PCA
import cv2

In [2]:
image_dir = "/Users/rfalcao/Documents/FYP/ManualSegmentationAnns/cusp_images"
mask_dir = "/Users/rfalcao/Documents/FYP/ManualSegmentationAnns/masks"

In [3]:


# Helper to normalize a vector
def normalize(v):
    return v / np.linalg.norm(v)

# Process one image-mask pair
def process_image(image_name):
    image_path = os.path.join(image_dir, image_name)
    mask_path = os.path.join(mask_dir, image_name.replace(".jpg", ".png"))

    # Load image and mask
    image = np.array(Image.open(image_path).convert("RGB"))
    mask = np.array(Image.open(mask_path).convert("L"))

    # Threshold mask to binary
    mask = (mask > 0).astype(np.uint8)

    # Get coordinates of mask pixels
    y_idxs, x_idxs = np.nonzero(mask)
    coords = np.stack((x_idxs, y_idxs), axis=1)  # shape: (N, 2)

    if coords.shape[0] < 2:
        print(f"Mask too small in {image_name}")
        return

    # Apply PCA
    pca = PCA(n_components=2)
    pca.fit(coords)

    # Get principal axis direction
    direction = pca.components_[0]  # (dx, dy)
    center = pca.mean_

    # Project all points onto the axis
    projections = coords @ direction
    tip_coord = coords[np.argmax(projections)]
    base_coord = coords[np.argmin(projections)]

    # Vector from base to tip
    vector = tip_coord - base_coord
    unit_vector = normalize(vector)

    print(f"{image_name}: unit vector = {unit_vector}")

    # --- Optional: visualization ---
    plt.imshow(image)
    plt.plot([base_coord[0], tip_coord[0]], [base_coord[1], tip_coord[1]], 'r-', linewidth=2)
    plt.scatter(center[0], center[1], color='blue', label='PCA Center')
    plt.arrow(base_coord[0], base_coord[1],
              unit_vector[0]*30, unit_vector[1]*30,
              head_width=5, color='lime', label='Direction')
    plt.title(f"{image_name} - PCA direction")
    plt.legend()
    plt.axis('off')
    plt.show()

# Example usage
for fname in os.listdir(image_dir)[:8]:
    if fname.endswith((".png", ".jpg", ".tif")):
        process_image(fname)


FileNotFoundError: [Errno 2] No such file or directory: '/Users/rfalcao/Documents/FYP/ManualSegmentationAnns/masks/cusp_254.jpg'